<a href="https://colab.research.google.com/github/AtogmatomaProgramming/OdonScanAI/blob/develop/manage_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#=== En caso de que no esté instalado scikit-learn ni tensorflow ===
!pip install scikit-learn
!pip install tensorflow

In [ ]:
#=== Importar librerias ===

import os
import json
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

In [ ]:
#=== Funciones de trabajo ===

# Función para obtener rutas de imágenes y etiquetas
def obtener_imagenes_y_etiquetas(carpeta_base, archivo_json):
    # Cargar o inicializar el JSON
    if os.path.exists(archivo_json):
        with open(archivo_json, 'r') as f:
            especies_dict = json.load(f)
    else:
        especies_dict = {}

    image_paths = []
    labels = []

    for especie in sorted(os.listdir(carpeta_base)):
        especie_dir = os.path.join(carpeta_base, especie)
        if os.path.isdir(especie_dir):
            if especie not in especies_dict:
                especies_dict[especie] = len(especies_dict)
            label = especies_dict[especie]

            for root, _, files in os.walk(especie_dir):
                for file in files:
                    if file.endswith(('.png', '.jpg', '.jpeg')):
                        image_paths.append(os.path.join(root, file))
                        labels.append(label)

    # Guardar el JSON actualizado
    with open(archivo_json, 'w') as f:
        json.dump(especies_dict, f)

    return image_paths, labels

# Función para preparar el dataset utilizando tf.data
def preparar_datasets(image_paths, labels, batch_size=32, validation_split=0.2):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Cargar y preprocesar las imágenes
    def cargar_y_preprocesar(ruta, etiqueta):
        imagen = tf.io.read_file(ruta)
        imagen = tf.image.decode_jpeg(imagen, channels=3)
        imagen = tf.image.resize(imagen, [224, 224])
        imagen = tf.image.convert_image_dtype(imagen, tf.float32)  # Normalizar
        return imagen, etiqueta

    dataset = dataset.map(cargar_y_preprocesar, num_parallel_calls=tf.data.AUTOTUNE)

    # Dividir en entrenamiento y validación
    dataset_size = len(image_paths)
    train_size = int((1 - validation_split) * dataset_size)

    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    # Configurar el dataset para batching, mezclado y cache
    train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds

# Definir el modelo
def crear_modelo(num_clases):
    modelo = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_clases, activation='softmax')
    ])

    modelo.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    return modelo



# Función principal
def main(carpeta_base, archivo_json, model_name):
    # Crear rutas de imágenes y etiquetas desde las carpetas de clases
    image_paths, labels = obtener_imagenes_y_etiquetas(carpeta_base, archivo_json)

    # Preparar los datasets de entrenamiento y validación
    train_ds, val_ds = preparar_datasets(image_paths, labels)

    # Crear el modelo con el número de clases detectado
    modelo = crear_modelo(num_clases=len(set(labels)))

    # Entrenar el modelo
    history = modelo.fit(train_ds, validation_data=val_ds, epochs=10)

    # Guardar el modelo entrenado
    modelo.save(model_name)



In [ ]:
#=== Ejecucion del script ===

carpeta_base = "/content/drive/My Drive/alpha_training/frames"
archivo_json= "/content/drive/My Drive/alpha_training/etiquetas.json"
model_name = "/content/drive/My Drive/alpha_training/modelo_gamma.h5"

main(carpeta_base, archivo_json, model_name)